# LSTM + GloVe + Cross-validation + LearningRate changes + ...
https://www.kaggle.com/demesgal/lstm-glove-lr-decrease-bn-cv-lb-0-047/notebook
# Notes
1) GRU is very similar to LSTM and not better

2) GloVe dimension is very important. I recommend to use GloVe 840b 300d if you can (it's very hard to use it in kaggle kernels)

3) Cross Validation is interesting for hiperparameters tuning, but for higher score you shoudn't use validation_split

4) First Epoch is very unstable. So I use small LR on first step

5) Dataset size is small. So you may use some additional datasets and then finetune model

6) It's hard not to overfit the model and I have n't found yet a good way to solve this problem. BatchNormalization/Dropout don't really help. 
https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/46494#263247
Maybe Reccurent Batch Normalization can help, but it is'not implemented in keras.

7) Use Attention layer from here (AttLayer):
https://github.com/dem-esgal/textClassifier/blob/master/textClassifierHATT.py

Thanks to (https://www.kaggle.com/CVxTz/keras-bidirectional-lstm-baseline-lb-0-051)

In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
print('hi')

Using TensorFlow backend.


hi


Glove dimension 100

In [2]:
#path = '../input/'
path ='C:/__KAGGLE__/ToxicCommentClassification'
#comp = 'jigsaw-toxic-comment-classification-challenge/'

#-----------------------------------
# http://ronny.rest/blog/post_2017_08_04_glove/
#-----------------------------------
#EMBEDDING_FILE=f'{path}glove-vectors/glove.6B.100d.txt'
#EMBEDDING_FILE=f'{path}glove6b50d/glove.6B.50d.txt'
EMBEDDING_FILE=f'{path}/glove.6B/glove.6B.50d.txt'

#TRAIN_DATA_FILE=f'{path}{comp}train.csv'
#TEST_DATA_FILE=f'{path}{comp}test.csv'
TRAIN_DATA_FILE=f'{path}/data_2018_02_16/train.csv/train.csv'
TEST_DATA_FILE=f'{path}/data_2018_02_16/test.csv/test.csv'

Set some basic config parameters:

In [3]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

Read in our data and replace missing values:

In [4]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [5]:
train.shape # (159571, 8)
type(train) # pandas.core.frame.DataFrame
train.head(n=1)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0


In [6]:
test.shape # (153164, 2)
test.head(n=1)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...


Standard keras preprocessing, to turn each comment into a list of word indexes of equal length (with truncation or padding as needed).

In [7]:
tokenizer = Tokenizer(num_words=max_features)  # max_features = 20000 = how many unique words to use
tokenizer.fit_on_texts(list(list_sentences_train)) # fit is done on training data only

In [8]:
# vectorization of train and test sentences
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [9]:
type(list_tokenized_train) # list
#list_tokenized_train[0:4]
type(X_t) #numpy.ndarray
X_t.shape # (159571, 100)
print("X_t[0,:]=",X_t[0,:], "\n    X_te[0,]=", X_te[0,])

X_t[0,:]= [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0   688    75     1   126   130   177    29
   672  4511 12052  1116    86   331    51  2278 11448    50  6864    15
    60  2756   148     7  2937    34   117  1221 15190  2825     4    45
    59   244     1   365    31     1    38    27   143    73  3462    89
  3085  4583  2273   985] 
    X_te[0,]= [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0  2665   655  8849   656
     8    57 16388    83   884   356    16  3222    76    21     6     4
  6865     6  1521     7    56   655  4942  1898   682  6908     4    96


Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

In [10]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
#----------------------------------------------------------------------------------------------
# https://stackoverflow.com/questions/9233027/unicodedecodeerror-charmap-codec-cant-decode-byte-x-in-position-y-character
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf8"))

Use these vectors to create our embedding matrix, with random initialization for words that aren't in GloVe. We'll use the same mean and stdev of embeddings the GloVe has when generating the random init.

In [11]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.020940498, 0.6441043)

In [12]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

Bidirectional LSTM with half-size embedding with two fully connected layers

In [13]:
DropOutRate = 0.1 #0.1
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=DropOutRate, recurrent_dropout=DropOutRate))(x)
x = GlobalMaxPool1D()(x)
x = BatchNormalization()(x)
x = Dense(50, activation="relu")(x)
#x = BatchNormalization()(x)
x = Dropout(DropOutRate)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)

import keras.backend as K
def loss(y_true, y_pred):
     return K.binary_crossentropy(y_true, y_pred)
    

In [14]:
model.compile(loss=loss, optimizer='nadam', metrics=['accuracy'])

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 50)           1000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 100)          40400     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
__________

Now we're ready to fit out model! Use `validation_split` when for hyperparams tuning

In [16]:
def schedule(ind):
    a = [0.002,0.003, 0.000]
    return a[ind]
lr = callbacks.LearningRateScheduler(schedule)
model.fit(X_t, y, batch_size=64, epochs=3, validation_split=0.1, callbacks=[lr])

#model.fit(X_t, y, batch_size=512, epochs=1, validation_split=0.1)
#model.fit(X_t, y, batch_size=64, epochs=3, callbacks=[lr])

Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 561s 4ms/step - loss: 0.0623 - acc: 0.8798 - val_loss: 0.0486 - val_acc: 0.9934
Epoch 2/3
143613/143613 [==============================] - 564s 4ms/step - loss: 0.0470 - acc: 0.9507 - val_loss: 0.0476 - val_acc: 0.9920
Epoch 3/3
143613/143613 [==============================] - 561s 4ms/step - loss: 0.0414 - acc: 0.9880 - val_loss: 0.0474 - val_acc: 0.9911


And finally, get predictions for the test set and prepare a submission CSV:

In [17]:
y_test = model.predict([X_te], batch_size=64, verbose=1)


153164/153164 [==============================] - 122s 797us/step


In [18]:
#sample_submission = pd.read_csv(f'{path}{comp}sample_submission.csv')
sample_submission = pd.read_csv(f'{path}/data_2018_02_16/sample_submission.csv/sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv(f'{path}/Ivan_0110/2018_02_16/v1/LSTM-submission_2018_02_17_v1.csv', index=False)

In [ ]:
ivan_mod_2018_02_17_config_v1 = model.get_config()

In [ ]:
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

# serialize model to JSON
model_json = model.to_json()
with open(f'{path}/data_2018_02_16/model_2018_02_16_v2.json', "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(f'{path}/data_2018_02_16/model_2018_02_16_v2.h5')
print("Saved model to disk")

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")